# Tentative de carte des résulats par BDV dans toute la France

In [ ]:
import geopandas as gpd
import pandas as pd 
import geoplot
import geoplot.crs as gcrs

In [ ]:
ResultatsCirco = pd.read_csv("resultats-par-niveau-cirlg-t1-france-entiere_v2.csv", encoding='iso-8859-1',delimiter=';') #source https://www.data.gouv.fr/fr/pages/donnees-des-elections/

In [ ]:
import json
import fiona
from shapely.geometry import shape 
#Read data
collection = list(fiona.open("circonscriptions2010.geoJSON",'r'))
df1 = pd.DataFrame(collection)

#Check Geometry
def isvalid(geom):
    try:
        shape(geom)
        return 1
    except:
        return 0
    
df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
Circo_Map = gpd.GeoDataFrame.from_features(collection)

In [ ]:
from shapely.geometry import MultiPolygon
Circo_Map2 = Circo_Map
def multi(geom):
    if geom.type=='GeometryCollection':
        return MultiPolygon([x for x in geom.geoms])
    else:
        return geom
    
Circo_Map2['geometry'] = Circo_Map2['geometry'].apply(lambda x: multi(x))

In [ ]:
ResultatsCirco2 = ResultatsCirco 

def tri(row): #Récupération des candidats NUPES
    if row['Nuance.1']=="NUP":
        return(row['% Voix/Exp.1'])
    if row['Nuance.2']=="NUP":
        return(row['% Voix/Exp.2'])
    if row['Nuance.3']=="NUP":
        return(row['% Voix/Exp.3'])
    if row['Nuance.4']=="NUP":
        return(row['% Voix/Exp.4'])
    if row['Nuance.5']=="NUP":
        return(row['% Voix/Exp.5'])
    if row['Nuance.6']=="NUP":
        return(row['% Voix/Exp.6'])
    
ResultatsCirco2['% Voix/Exp'] = ResultatsCirco2.apply(lambda row: tri(row), axis=1)
ResultatsCirco2 =ResultatsCirco2[~ResultatsCirco2['Code du département'].str.contains('Z', na=False)]
ResultatsCirco2 = ResultatsCirco2[['Code du département','Code de la circonscription', '% Voix/Exp','% Abs/Ins']] 

In [ ]:
ResultatsCirco2['name']=ResultatsCirco2['Code du département'].apply(str).str.zfill(2)+ResultatsCirco2['Code de la circonscription'].apply(str).str.zfill(3)
ResultatsCirco2=ResultatsCirco2.drop(['Code du département','Code de la circonscription'], axis=1)

In [ ]:
ResultatsCirco_Geo = pd.merge(Circo_Map2,ResultatsCirco2,how='inner',on='name') #Carte par circonscription et non BDV

In [ ]:
ResultatsCirco_Geo[ResultatsCirco_Geo['% Abs/Ins'].isna()]

In [ ]:
import matplotlib as plt
ax = geoplot.webmap(ResultatsCirco_Geo,projection=gcrs.WebMercator(), linewidth=.3,zoom=7, figsize=(120, 120))
geoplot.choropleth(ResultatsCirco_Geo,projection=gcrs.WebMercator(),hue='% Abs/Ins',linewidth=.3, legend=True, 
                   legend_kwargs={'location': 'bottom','anchor' : (0.8, 2.2), 'shrink' : 0.25,'label':'Abstention', 'aspect':30},ax=ax
                  )
plt.pyplot.savefig('yolo3.png')